<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#ES算法" data-toc-modified-id="ES算法-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>ES算法</a></span><ul class="toc-item"><li><span><a href="#ES算法在GR系统上的应用" data-toc-modified-id="ES算法在GR系统上的应用-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>ES算法在GR系统上的应用</a></span></li><li><span><a href="#非场景化ES" data-toc-modified-id="非场景化ES-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>非场景化ES</a></span></li><li><span><a href="#场景化ES" data-toc-modified-id="场景化ES-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>场景化ES</a></span></li></ul></li><li><span><a href="#ES算法在GR系统上的应用" data-toc-modified-id="ES算法在GR系统上的应用-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>ES算法在GR系统上的应用</a></span></li><li><span><a href="#非场景化ES" data-toc-modified-id="非场景化ES-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>非场景化ES</a></span></li><li><span><a href="#场景化ES" data-toc-modified-id="场景化ES-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>场景化ES</a></span></li><li><span><a href="#推荐阅读" data-toc-modified-id="推荐阅读-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>推荐阅读</a></span></li></ul></div>

## ES算法

> **推荐阅读**⭐️⭐️⭐️⭐️⭐️
> 1. Evolution Strategies as a Scalable Alternative to Reinforcement Learning
> 2. The CMA Evolution Strategy: A Tutorial
> 3. Guided evolutionary strategies: Augmenting random search with surrogate gradients

ES算法是一种进化策略，通过不断的前向进化，可以达到和强化学习相似的效果。ES算法通过扰动神经网络中的参数,不同的参数带来不同的reward,通过reward大小对应的参数来按一定方向和比例更新原始的参数。ES算法有以下几个特点：
1. ES算法计算简单高效，可高度并行，资源消耗小，使用较少代价的训练达到和DQN等RL模型相似的效果
2. ES算法还可以推广到其他模块、其他模型的优化，不受参数作用方式、模块位置以及作用长度的限制，比如通过gr获得的reward，来进行粗排、GRC等阶段的参数优化等
3. 不依赖于误差反向传播，ES只依赖最终的反馈，适用于长流程复杂目标优化

ES进化示意图如下，白点代表当前模型内部参数， 周围的黑点是由白点抖动得到的噪声参数，对所有的噪声参数根据奖励值（红色=高、蓝色=低）计算一个回报向量，根据这些回报向量（白色箭头为梯度）。通过不断的参数进化，可以收敛到最优值。

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20240624234005101.png" alt="image-20240624234005101" style="zoom:50%; display: block; margin: auto;" />

伪代码如下：
```
1. 设置初始参数向量w
2. 设置es算法轮数n（更新n轮后结束，后续会修改收敛条件）
3. for t = 0, 1, 2, ..., n
  (1) 计算当前基线 reward: rbase
  (2) step_list = []
  (3) 生成 m 组扰动向量 e1, e2, ..., em，对每一个 e 生成 wi = w + ei
  (4) 对每一个 wi，计算 reward(ri) 和 deta_ri = ri - rbase
  (5) step_list.append(ei * deta_ri)
  (6) 移动步长 step = sum(step_list) / len(step_list) * learning_rate
  (7) 更新 w: w = w + step
  (8) 计算新的 w 下的 reward_new，更新  reward = reward_new
```

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20240624234218407.png" alt="image-20240624234218407" style="zoom:33%; display: block; margin: auto;" />

### ES算法在GR系统上的应用

GR线上系统较为复杂，同时存在较多超参数，而这些超参数难以以某一目标有效进行离线监督学习，非常符合作用流程长、策略系统复杂等特点。因此尝试利用ES对GR线上超参数进行一次性寻优。这种方法有以下几个特征：
1. 不过度依赖经验调优：所有超参同时寻优，避免了需要针对不同参数、不同作用方式需要不同调节经验的问题。
2. 全局最优：众多参数同时调优有利于超参数整体达到最优，更易找到比单独寻优更全局最优的可能。
3. 在线进化、快速自适应：在线进化有利于系统能够根据变化快速自适应，避免系统变化以后很多参数过时的问题。
4. 可扩展性强：能够推广到其他模块、其他模型的参数及超参数优化，不受参数作用方式、模块位置和作用长度的限制。

### 非场景化ES

现有 feed 推荐策略，在 GR 序列生成阶段会根据点展比(ctr)和预估时长(dur)进行点击时长融合排序。现有的排序公式为 ctr * dur + xgb_dur，本策略使用 ctr、dur 的多项式形式来进行融合排序，扩展了搜索空间，同时在多项式系数的寻优中，使用ES(Evolution Strategies)算法进行迭代进化，以期获取更优的融合效果。

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20240625002054845.png" alt="image-20240625002054845" style="zoom:66%; display: block; margin: auto;" />

上图为新融合公式，采用(ctr,dur)的多项式二次展开形式，原有的融合公式为新融合公式的子集，新公式扩展了参数的搜索空间，有利于搜到更优的结果。

先圈定1%的小流量用户，对参数进行初始化，通过在参数上添加随机扰动的噪声，在线得到该参数下用户的reward，离线收集样本，利用ES算法进行迭代参数。

### 场景化ES

建立一个基于场景化特征的线性融合模型，通过ES进化学习不同用户在各场景下的个性化与场景化融合参数，优化不同场景下的点击融合效果，进而提升推荐效果。

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20240625002314531.png" alt="image-20240625002314531" style="zoom:50%; display: block; margin: auto;" />

使用刷新类型、是否是低活用户、refresh_state、refresh_typr、net(网络状态)  五维场景化特征，通过一个线性模型来得到二项展开式的参数，使用ES算法对模型参数(vector_w及bias)进行寻优。

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20240625002458037.png" alt="image-20240625002458037" style="zoom:50%; display: block; margin: auto;" />

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20240625002549877.png" alt="image-20240625002549877" style="zoom:50%; display: block; margin: auto;" />